TEXT

In [34]:
# IMPORTS
import random
import numpy as np
import pandas as pd

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator, HillClimbSearch, BicScore, K2Score
from pgmpy.sampling import BayesianModelSampling

In [7]:
# Set random seed to ensure reproducibility
random.seed(37)

In [138]:
file_path = 'instances_01_KP/low-dimensional/f1_l-d_kp_10_269'
# file_path = 'instances_01_KP/low-dimensional/f2_l-d_kp_20_878'
# file_path = 'instances_01_KP/large_scale/knapPI_1_100_1000_1'

data = np.loadtxt(file_path, dtype=int, usecols=(0, 1))
col_1 = data[:, 0]
col_2 = data[:, 1]

n_items = col_1[0]
capacity = col_2[0]

values = data[1:, 0]
weights = data[1:, 1]

# print("First Column:", col_1)
# print("Second Column:", col_2)
print("number of items:", n_items)
print("max weight:", capacity)
print("values:", values)
print("weights:", weights)

number of items: 10
max weight: 269
values: [55 10 47  5  4 50  8 61 85 87]
weights: [95  4 60 32 23 72 80 62 65 46]


In [9]:
def knapsack_fitness(solution, values, weights, capacity):
    total_weight = np.dot(solution, weights)
    total_value = np.dot(solution, values)
    if total_weight > capacity:
        return 0  # Invalid solution
    return total_value

In [10]:
def generate_random_solution(length):
    return np.random.randint(2, size=length)

In [11]:
test_sol = generate_random_solution(n_items)
test_sol_fitness = knapsack_fitness(test_sol, values, weights, capacity)
print(test_sol)
print(test_sol_fitness)

[1 1 0 1 1 1 1 1 1 1]
0


In [143]:
# Store problem items in dictionary

items = {}
for i in range(n_items):
    items[i] = (values[i], weights[i])

print(items)
print(len(items))
print(items[3][1])

{0: (55, 95), 1: (10, 4), 2: (47, 60), 3: (5, 32), 4: (4, 23), 5: (50, 72), 6: (8, 80), 7: (61, 62), 8: (85, 65), 9: (87, 46)}
10
32


In [145]:
# Initialise fitness
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
# weights is positive to signify maximisation of fitness, use negative to minimise
# weights must be a tuple to allow to single and multi objective problems
# to be treated in the same way

# Initialise individual
creator.create("Individual", list, fitness=creator.FitnessMax)

IND_SIZE = len(items) # number of genes in individual

def evalIndividual(individual):
    weight = 0
    value = 0
    # print(individual)
    for i in range(IND_SIZE):
        value += items[i][0] * individual[i]
        weight += items[i][1] * individual[i]
    if weight > capacity:
        return (0,)
    # print(value)
    return (value,)

toolbox = base.Toolbox()
toolbox.register("attr_int", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evalIndividual)
toolbox.register("select", tools.selBest)

def fit_bayesian_network(population):
    # convert population to data for model
    data = np.array(population)
    df = pd.DataFrame(data)

    # dummy_edges = [(0, node) for node in range(IND_SIZE) if node != 0]

    # learn structure of bayesian network
    hc = HillClimbSearch(df)

    # determine structure of model
    # model_structure = hc.estimate(scoring_method=BicScore(df), fixed_edges=dummy_edges)
    model_structure = hc.estimate(scoring_method=K2Score(df))

    # missing_nodes = set(range(IND_SIZE)) - set(model_structure.nodes())
    # print(set(range(IND_SIZE)))
    # print(set(model_structure.nodes()))
    # print(f'missing nodes {missing_nodes}')
    # print(f'nodes {model_structure.nodes()}')

    # print(f"Data shape: {df.shape}")
    # print(f"Data columns: {df.columns}")

    # for node in range(IND_SIZE):
    #     if node != 0 and node not in model_structure.nodes():  # Prevent self-loops and add only if missing
    #         model_structure.add_edge(0, node)

    # create model using best structure
    model = BayesianNetwork(model_structure.edges())

    for node in range(IND_SIZE):
        if node != 0 and node not in model.nodes():
            model.add_edge(0, node)

    # learn parameters and return model
    model.fit(df, estimator=MaximumLikelihoodEstimator)

    # print(f'nodes {model.nodes()}')
    print(f'edges {model.edges()}')
    # for cpd in model.get_cpds():
        # print(cpd)
    return model

def sample_population_from_bayesian(bayesian_network, pop_size):
    # generate individuals by sampling bayesian netwrok
    inference = BayesianModelSampling(bayesian_network)
    new_population = inference.forward_sample(size=pop_size)
    # print(f'new pop shape {new_population.shape}')
    new_population = new_population.values.tolist()
    return new_population

def BOA(pop_size, n_generations, n_selected):
    gen = 0
    # initialise population
    population = toolbox.population(pop_size)
    # print(population)

    # loop through each generation
    for generation in range(n_generations):
        fitnesses = list(map(toolbox.evaluate, population)) # eval pop

        for ind, fit in zip(population, fitnesses):
            # print(fit)
            ind.fitness.values = fit
        
        # print best in population
        top_individual = tools.selBest(population, 1)[0]
        print(f"Generation {gen}: Best Individual = {top_individual}, Fitness = {top_individual.fitness.values}")
        gen += 1

        # select best individuals from population
        selected_population = toolbox.select(population, n_selected)

        # learn bayesian network from selected individuals
        network = fit_bayesian_network(selected_population)

        # generate new population by sampling bayesian network
        new_population = sample_population_from_bayesian(network, len(population))

        # replace population
        population[:] = [creator.Individual(ind) for ind in new_population]


In [146]:
BOA(100, 20, 30)

Generation 0: Best Individual = [0, 0, 1, 0, 1, 0, 0, 1, 1, 1], Fitness = (284.0,)


  0%|          | 14/1000000 [00:00<10:51:10, 25.59it/s]


edges [(0, 5), (0, 2), (3, 2), (3, 4), (3, 5), (4, 0), (6, 1), (6, 7), (7, 0), (7, 5), (8, 3), (8, 1), (8, 2), (9, 8)]


Generating for node: 2: 100%|██████████| 10/10 [00:00<00:00, 434.79it/s]


Generation 1: Best Individual = [0, 0, 1, 1, 0, 0, 0, 1, 1, 1], Fitness = (285.0,)


...

Generating for node: 9: 100%|██████████| 10/10 [00:00<00:00, 476.24it/s]


Generation 19: Best Individual = [0, 1, 1, 0, 1, 1, 0, 0, 0, 1], Fitness = (198.0,)


  0%|          | 0/1000000 [00:00<?, ?it/s]


edges [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9)]


Generating for node: 9: 100%|██████████| 10/10 [00:00<00:00, 526.41it/s]
